In [1]:
import os 
import pickle
import time 
import random
import numpy as np 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from save_data import save
from data_splitter import get_train_and_test_data

In [2]:
level = 'SeeSaw'
save(level, debug=True, delete=True)
os.chdir('D:/Documents/Python/fallguy_bot/DATA/LEVELS/'+ level)
X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
X = X/255.0



File to delete Not found
Reading: D:/Documents/Python/fallguy_bot/DATA/LEVELS/SeeSaw
Run # 1
Dead categories= 501
Fine categories= 1124
Raw  categories= 1625
raw  img list= 1625
Fine img list= 1124
Run # 2
Dead categories= 901
Fine categories= 2261
Raw  categories= 3162
raw  img list= 3162
Fine img list= 2261
Run # 3
Dead categories= 1011
Fine categories= 2806
Raw  categories= 3817
raw  img list= 3817
Fine img list= 2806
Run # 4
Dead categories= 460
Fine categories= 1046
Raw  categories= 1506
raw  img list= 1506
Fine img list= 1046
Run # 5
Dead categories= 662
Fine categories= 1710
Raw  categories= 2372
raw  img list= 2372
Fine img list= 1710
Run # 6
Dead categories= 665
Fine categories= 1749
Raw  categories= 2414
raw  img list= 2414
Fine img list= 1749
Run # 7
Dead categories= 636
Fine categories= 1586
Raw  categories= 2222
raw  img list= 2222
Fine img list= 1586
Run # 8
Dead categories= 898
Fine categories= 2261
Raw  categories= 3159
raw  img list= 3159
Fine img list= 2261
Run # 9
De

In [4]:
y.shape

(34946, 7)

In [9]:
X_train, y_train, X_test, y_test = get_train_and_test_data(X,y)

get max index
get index for test
getting index for train
creating X_test
creating y test
creating X_train
creating y_train
try to return


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(31452, 427, 240, 1)
(31452, 7)
(3494, 427, 240, 1)
(3494, 7)


In [ ]:
X_train = X_train.astype("float32")
y_train = y_train.astype("float32")

X_test = X_test.astype("float32")
y_test = y_test.astype("float32")


In [26]:
num_counter = 0
double_key = []
category_counter = -1
space_keys = []
w_counter = -1
pos_of_key = 0
for category in  y_test:
    category_counter += 1
    for num in category:
        w_counter += 1 
        if num == 1:
            num_counter += 1
        if w_counter == pos_of_key and num == 1:
            space_keys.append(category_counter)
        
            
    if num_counter == 2:
        double_key.append(category_counter)
    num_counter = 0
    w_counter = -1
    
        
        
    


In [27]:
print('persent of double key is =', "%.2f" %round((len(double_key)*100/len(y_test)),2))
print('Have ',len(space_keys),'keys','Its have ', "%.2f" %round((len(space_keys)*100/len(y_test)),2),'Percent of DF')

persent of double key is = 39.75
Have  2434 keys Its have  69.66 Percent of DF


In [22]:
model = Sequential()

model.add(Conv2D((64), (2,2), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D((32), (2,2)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(8))
model.add(Activation('relu'))


model.add(Dense(7))
model.add(Activation('sigmoid'))

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy'],
)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 426, 239, 64)      320       
_________________________________________________________________
activation_35 (Activation)   (None, 426, 239, 64)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 213, 119, 64)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 212, 118, 32)      8224      
_________________________________________________________________
activation_36 (Activation)   (None, 212, 118, 32)      0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 106, 59, 32)       0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 200128)           

In [23]:
NAME = 'CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [24]:
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_test, y_test), callbacks=[tensorboard])

Epoch 1/5
492/492 [==============================] - 1674s 3s/step - loss: 0.6219 - accuracy: 0.7283 - val_loss: 0.5551 - val_accuracy: 0.7393
Epoch 2/5
492/492 [==============================] - 1681s 3s/step - loss: 0.5099 - accuracy: 0.7335 - val_loss: 0.4714 - val_accuracy: 0.7393
Epoch 3/5
492/492 [==============================] - 1626s 3s/step - loss: 0.4434 - accuracy: 0.7335 - val_loss: 0.4192 - val_accuracy: 0.7393
Epoch 4/5
492/492 [==============================] - 1584s 3s/step - loss: 0.4007 - accuracy: 0.7335 - val_loss: 0.3849 - val_accuracy: 0.7393
Epoch 5/5
492/492 [==============================] - 1577s 3s/step - loss: 0.3721 - accuracy: 0.7335 - val_loss: 0.3614 - val_accuracy: 0.7393


In [25]:
model.save('cnn.model')

INFO:tensorflow:Assets written to: cnn.model\assets
